# formata-texto
Esse script recebe arquivos .txt extraídos do site da Câmara dos Deputados e executa operações de limpeza e formatação usando regex.

#### Importação de pacotes

In [1]:
import glob
import os
import pandas as pd
import re
from unidecode import unidecode

#### Ler arquivos externos

In [2]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

In [3]:
def make_df(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [ open(file).read() for file in file_list ]
    file_list = [ unidecode(file) for file in file_list ]
    
        
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'     : [ file for file in file_list ],
        'CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE' : [ re.search("\-([A-Z\s]+)\-", file).group(1) for file in file_list ],
        'SESSION_DATE' : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")

    return df

#### Funções de formatação e busca usando regex

In [4]:
def find_speakers(string):
    
    '''
    Essa função detecta o padrão de texto
    que antecede a fala de um deputado e
    retorna um objeto match (via re.find_all).
    Ele é útil para detectar QUANTOS deputados
    falaram em determinada string textual.
    '''
    
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    return matches

In [5]:
def extract_full_quote(string):
    
    '''
    Essa função extrai todas as falas de Jair Bolsonaro
    em uma determinada string. O pattern de regex encontra,
    primeiro, uma fala qualquer de Jair Bolsonaro. Então, pega
    tudo que está entre essa fala e a fala de outro parlamentar 
    ou o fim do arquivo. Isso é necessário porque há arquivos
    que misturam a fala de vários parlamentares, geralmente
    quando estão envolvidos em uma discussão.
    '''
    
    pattern  = "O?\s+SR\.?\s+JAIR\s+BOLSONARO\s?(\((Bloco\/)?\w{2,}\s?\-\s?\w{2}[^)]+\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    full_quote = [ ]
    for match in matches:
        quote = match[2]
        quote = ' '.join(quote.split()) # Remove espaços múltiplos internos usando a operação join de lista
        quote = quote.replace("- ", "")
        full_quote.append(quote)
        
    full_quote = ' [ INTERRUPÇÃO ] '.join(full_quote)
    return full_quote

#### Funções para aplicar operações de regex no dataframe

In [6]:
def apply_count_speakers(row):
    
    '''
    Aplica, linha a linha, a função
    find_speakers(string)
    '''
    
    matches = find_speakers(row.CONTENT)
    speaker_count = len(matches)
    
    return pd.Series({ "SPEAKER_COUNT":speaker_count 
        })

In [15]:
def apply_extract_full_quote(row):
    
    '''
    Aplica, linha a linha, a função
    extract_full_quote(string)
    '''
    
    full_quote = extract_full_quote(row.CONTENT)
    return pd.Series({
        "FULL_QUOTE" : full_quote
    })

In [16]:
jb = make_df( find_fpaths("../data/txts/plenario/", "*.txt") )

In [17]:
jb["SPEAKER_COUNT"] = jb.apply(apply_count_speakers, axis=1)

In [18]:
jb["FULL_QUOTE"] = jb.apply(apply_extract_full_quote, axis=1)

In [23]:
jb[jb.FULL_QUOTE == ""]

,CONTENT,FILE,SESSION_DATE,SESSION_TYPE,SPEAKER_COUNT,FULL_QUOTE
73,Ata da 110a (centesima decima) Sessao da Camar...,../data/txts/plenario/10052017-9h30-HOMENAGEM-...,2017-05-10,HOMENAGEM,0,
105,Ata da 117a (centesima decima setima) Sessao d...,../data/txts/plenario/16052017-9h30-HOMENAGEM-...,2017-05-16,HOMENAGEM,0,
114,Ata da 376a (tricentesima septuagesima sexta) ...,../data/txts/plenario/19112013-10h00-HOMENAGEM...,2013-11-19,HOMENAGEM,0,
130,O SR. JAIR BOLSONARO,../data/txts/plenario/06022012-14h24-PEQUENO E...,2012-02-06,PEQUENO EXPEDIENTE,1,
131,Ata da 154a (centesima quinquagesima quarta) S...,../data/txts/plenario/06062013-10h00-HOMENAGEM...,2013-06-06,HOMENAGEM,0,
168,Ata da 123a (centesima vigesima terceira) Sess...,../data/txts/plenario/19052016-9h00-HOMENAGEM-...,2016-05-19,HOMENAGEM,0,
174,Ata da 60a (sexagesima) Sessao da Camara dos D...,../data/txts/plenario/30032016-9h00-HOMENAGEM-...,2016-03-30,HOMENAGEM,0,
297,O SR. DEPUTADO JAIR BOLSONARO - Sr. Presidente...,../data/txts/plenario/14092016-12h28-COMISSAO ...,2016-09-14,COMISSAO GERAL,4,
302,O SR. PRESIDENTE (Jair Bolsonaro) - Quero regi...,../data/txts/plenario/21052015-14h12-PEQUENO E...,2015-05-21,PEQUENO EXPEDIENTE,1,
305,Ata da 229a (ducentesima vigesima nona) Sessao...,../data/txts/plenario/15102014-9h30-HOMENAGEM-...,2014-10-15,HOMENAGEM,0,


In [ ]:
# TO DO

# Adicionar captura dos casos em que Jair Bolsonaro foi presidente da Câmara

In [22]:
for item in jb.FULL_QUOTE:
    print(item)
    print("***")
    print()

Sr. Presidente, Sras. e Srs. Deputados, primeiro, quero saudar os colegas da Policia Ferroviaria Federal presentes na Casa. Sras. e Srs. Deputados, ontem a noite participei, ao vivo, do Programa do Ratinho, e acho que fiz entender o motivo pelo qual tenho participado de debates contra o avanco homossexual nas escolas do ensino basico. Esteve comigo o Sr. Toni Reis, Presidente da Associacao Brasileira de Gays, Lesbicas, Bissexuais, Travestis e Transexuais ABGLT e um dos que elaboraram o material didatico que sera distribuido nas escolas para, segundo eles, combater a homofobia. Na verdade, esse material didatico estimula o homossexualismo e tudo o que podemos imaginar de ruim dentro das escolas. Fazem parte desse material inclusive alguns videos. Alias, ontem o Ratinho colocou no ar um deles, cujo titulo e Encontrando Bianca, no qual um garoto chamado Ricardo faz questao de ser chamado de Bianca, pinta as unhas, quer frequentar o banheiro feminino, namorar na escola e por ai afora. Exib

***

Sr. Presidente, parabenizo o Deputado Antonio Cambraia, companheiro que tambem labutou no Exercito durante muitos anos, pelo seu discurso, e gostaria, se me permite, de incorpora-lo ao meu. Parabenizo tambem o Deputado Joao Fontes, ex-petista, por estar agora numa posicao que nao esperava. Deputado Inocencio Oliveira, com a saida de alguns Parlamentares do PT, a unica vantagem para o partido e o aumento da concentracao de trairas por metro cubico. Sairam os Deputados coerentes e ficaram os trairas. Lamento tudo que vem acontecendo. Atraves do AI-5, cassavam-se, na maioria das vezes, corruptos; o 171 do PT cassa pessoas coerentes. Sr. Presidente, o ano esta findando. Nao farei aqui nenhum balanco. Talvez esta seja a ultima vez em que ocupo a tribuna desta Casa, mas devo dizer que o Governo do PT foi uma desgraca no corrente ano. As medidas adotadas pelo partido desestimularam os cidadaos a se empenharem em estudar, porque todos serao iguais no final da carreira, a exemplo dos servi